# Scraper Utility to obtain data from twse website
## References
- https://bitbucket.opst.c1.vanguard.com/users/uc8c/repos/gidm/browse/fdm1/fdm1.py
- https://www.twse.com.tw/en/page/trading/exchange/BWIBBU_d.html - P/E, P/B ratios
- https://www.twse.com.tw/en/page/trading/exchange/TWTASU.html - short sale vol
- https://www.twse.com.tw/en/page/trading/exchange/MI_INDEX.html#subtitle7 - daily summary - all
- https://www.twse.com.tw/en/page/trading/exchange/TWT85U.html - full delivery securities
- https://emops.twse.com.tw/server-java/t58query#
- https://app2.msci.com/eqb/custom_indexes/tw_performance.html - MSCI index constituents


In [48]:
urls = {
    "ratios" : 'https://www.twse.com.tw/en/page/trading/exchange/BWIBBU_d.html',
    "short_sales" : 'https://www.twse.com.tw/en/page/trading/exchange/TWTASU.html',
    "summary_px_vol" : 'https://www.twse.com.tw/en/page/trading/exchange/MI_INDEX.html#subtitle7',
    "full_delivery": 'https://www.twse.com.tw/en/page/trading/exchange/TWT85U.html'
}

In [110]:
#for Chrome bot stuff
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver import ActionChains
import time
import pandas as pd
import shutil
import glob
import re

In [3]:
pd.options.display.max_rows = 2000

In [4]:
def createWebDriver():
    """ creates the Chrome web driver bot """
    chrome_path = os.getcwd()+"\\chromedriver.exe"
    #chrome_path = "C:\\Users\\uc8c\\OneDrive - Vanguard\\Repos on Desktop - Laptop Migration 6-26-21\\GIDM\\gidm\\fdm1\\drivers"
    chrome_options = Options()
    #chrome_options.add_argument("--headless") 
    driver = webdriver.Chrome(executable_path = chrome_path, chrome_options=chrome_options)

    return driver

In [5]:
def get_full_delivery_data(driver, start_date='1/1/2022', end_date='1/5/2022'):
    """ Used to obtain full delivery data from the twse website """
   
    driver.get(urls['full_delivery'])
    
    date_range = pd.bdate_range(start=start_date, end=end_date)

    for d in date_range:
        #select month
        select_mo = Select(driver.find_element_by_xpath('//*[@id="d1"]/select[2]'))
        select_mo.select_by_index(d.month-1)
        time.sleep(1)

        #select day
        select_day = Select(driver.find_element_by_xpath('//*[@id="d1"]/select[3]'))
        select_day.select_by_index(d.day-1)
        time.sleep(1)

        #click query button
        driver.find_element_by_tag_name('a.button.search').click()
        time.sleep(4)

        #click download csv button
        driver.find_element_by_xpath('//*[@id="reports"]/div[1]/a[2]').click()
        time.sleep(4)

In [6]:
def get_short_sales_data(driver, start_date='1/1/2022', end_date='1/5/2022'):
    """ Used to obtain short sales data from twse website """
   
    driver.get(urls['short_sales'])
    
    date_range = pd.bdate_range(start=start_date, end=end_date)

    for d in date_range:
        #select month
        select_mo = Select(driver.find_element_by_xpath('//*[@id="d1"]/select[2]'))
        select_mo.select_by_index(d.month-1)
        time.sleep(1)

        #select day
        select_day = Select(driver.find_element_by_xpath('//*[@id="d1"]/select[3]'))
        select_day.select_by_index(d.day-1)
        time.sleep(1)

        #click query button
        driver.find_element_by_tag_name('a.button.search').click()
        time.sleep(4)

        #click download csv button
        driver.find_element_by_xpath('//*[@id="reports"]/div[1]/a[2]').click()
        time.sleep(4)

In [46]:
def get_ratios_data(driver, start_date='1/1/2022', end_date='1/5/2022'):
    """ Used to obtain ratios data from twse website """
   
    driver.get(urls['ratios'])
    date_range = pd.bdate_range(start=start_date, end=end_date) 

    for d in date_range:
        #select month
        select_mo = Select(driver.find_element_by_xpath('//*[@id="d1"]/select[2]'))
        select_mo.select_by_index(d.month-1)
        time.sleep(1)

        #select day
        select_day = Select(driver.find_element_by_xpath('//*[@id="d1"]/select[3]'))
        select_day.select_by_index(d.day-1)
        time.sleep(1)

        #click query button
        driver.find_element_by_tag_name('a.button.search').click()
        time.sleep(3)

        try:
            #click download csv button
            driver.find_element_by_xpath('//*[@id="reports"]/div[1]/a[2]').click()
        except:
            print(f"Error downloading data for: {d}")
        time.sleep(3)

In [49]:
def get_summary_data(driver, start_date='1/1/2022', end_date='1/5/2022'):
    """ Used to obtain summary data from twse website """
   
    driver.get(urls['summary_px_vol'])
    date_range = pd.bdate_range(start=start_date, end=end_date) 

    for d in date_range:
        #select month
        select_mo = Select(driver.find_element_by_xpath('//*[@id="d1"]/select[2]'))
        select_mo.select_by_index(d.month-1)
        time.sleep(1)

        #select day
        select_day = Select(driver.find_element_by_xpath('//*[@id="d1"]/select[3]'))
        select_day.select_by_index(d.day-1)
        time.sleep(1)

        #select category
        select_cat = Select(driver.find_element_by_xpath('//*[@id="main-form"]/div/div/form/select'))
        select_cat.select_by_visible_text('All(no Warrant & CBBC & OCBBC)')

        #click query button
        driver.find_element_by_tag_name('a.button.search').click()
        time.sleep(3)

        try:
            #click download csv button
            driver.find_element_by_xpath('//*[@id="reports"]/div[1]/a[2]').click()
        except:
            print(f"Error downloading data for: {d}")
        time.sleep(3)

In [8]:
################set up Chrome driver#######
try:   
    driver = createWebDriver()
    print("Successfully created Chrome bot")
    time.sleep(1)

except:
    print("Unexpected error:", sys.exc_info()[0])
    #time.sleep(1*60)
    raise

C:\Users\uc8c\AppData\Local\Temp/ipykernel_30916/1570040323.py:7: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path = chrome_path, chrome_options=chrome_options)


Successfully created Chrome bot


In [83]:
#run to obtain full delivery data, via downloaded csv files
get_full_delivery_data(driver, start_date='1/2/2022', end_date='1/5/2022')

In [86]:
#make zip file
shutil.make_archive("taiwan-watchlist-data-website-raw-01012022-10312022", 'zip', './data/')

'c:\\Users\\uc8c\\Desktop\\daa.eq-service-prefunding-challenge\\taiwan-watchlist-data-website-raw-01012022-10312022.zip'

In [99]:
#run to obtain short sales data, via downloaded csv files
#(NT$, trading unit/thousand shares)
#use Margin Short Sales column
get_short_sales_data(driver, start_date='5/2/2022', end_date='10/31/2022')

In [100]:
#make zip file
shutil.make_archive("taiwan-shortsale-data-website-raw-01012022-10312022", 'zip', './data/')

'c:\\Users\\uc8c\\Desktop\\daa.eq-service-prefunding-challenge\\taiwan-shortsale-data-website-raw-01012022-10312022.zip'

In [45]:
get_ratios_data(driver, start_date='1/27/2022', end_date='10/31/2022')

In [90]:
#make zip file
shutil.make_archive("all_data", 'zip', './data')

'c:\\Users\\uc8c\\Desktop\\daa.eq-service-prefunding-challenge\\all_data.zip'

In [51]:
get_summary_data(driver, start_date='1/6/2022', end_date='10/31/2022')

Error downloading data for: 2022-01-27 00:00:00
Error downloading data for: 2022-01-28 00:00:00
Error downloading data for: 2022-01-31 00:00:00
Error downloading data for: 2022-02-01 00:00:00
Error downloading data for: 2022-02-02 00:00:00
Error downloading data for: 2022-02-03 00:00:00
Error downloading data for: 2022-02-04 00:00:00
Error downloading data for: 2022-02-28 00:00:00
Error downloading data for: 2022-04-04 00:00:00
Error downloading data for: 2022-04-05 00:00:00
Error downloading data for: 2022-05-02 00:00:00
Error downloading data for: 2022-06-03 00:00:00
Error downloading data for: 2022-09-09 00:00:00
Error downloading data for: 2022-10-10 00:00:00


In [88]:
#make zip file
shutil.make_archive("taiwan-stockquotes-data-website-raw-010122-10312022", 'zip', './data')

'c:\\Users\\uc8c\\Desktop\\daa.eq-service-prefunding-challenge\\taiwan-stockquotes-data-website-raw-010122-10312022.zip'

In [7]:
summary_path = os.path.join(os.getcwd(), "other_data", "taiwan-summary-website-raw-01012022-10312022")

In [61]:
summary_path

'c:\\Users\\uc8c\\Desktop\\daa.eq-service-prefunding-challenge\\data\\taiwan-summary-website-raw-01012022-10312022'

In [63]:
summary_files = glob.glob(os.path.join(summary_path, '*'))

In [79]:
#extacting just the stock quotes from the summary files.
for filepath in summary_files[5:]:
    data_start = 1000000
    with open(filepath, 'r') as f:
        with open(os.path.join(os.path.split(filepath)[0], os.path.split(filepath)[1][:-4]+"_stock_quotes.csv"), 'w') as temp:
            for i, line in enumerate(f):
                try:
                    if "Security Code" in line:
                        data_start = i
                        temp.write(line)

                    if (i > data_start):
                        if "Remarks:" in line:
                            break
                        temp.write(line)
                        
                except:
                    print(f"Error reading line: {i}")
                    errors.append(i)


In [153]:
info = {
        "watchlist" : {
                        "globpath" : "./all_data/taiwan-watchlist-data-website-raw-010122-10312022/*",
                        "cols_to_drop" : ['Unnamed: 2'],
                        "skiprows" : 1,
                        "skipfooter": 4,
                        "date_parse": (-12,-4),
                        "usecols": None,
                        "dtypes": {'Security Code':"object", 'Periodic Call Auction Trading (see Remark below)':"object",'file_date':pd.Timestamp}
                },
    
        "shortsales" : {
                "globpath" : "./all_data/taiwan-shortsale-data-website-raw-01012022-10312022/*",
                "cols_to_drop": None,
                "usecols": [0,1,2],
                "skiprows" : 2,
                "skipfooter": 4,
                "date_parse": (-12,-4),
                "dtypes" : {'Security Code': "object", "Trading Volume": "Int64", "Trading Value": "Int64", "file_date": pd.Timestamp}
        },

        "ratios" :{
                "globpath" : "./all_data/taiwan-pe-pb-ratios-website-raw-01012022-10312022/*",
                "cols_to_drop": None,
                "skiprows" :1, 
                "skipfooter" : 2, 
                "usecols" : [0,1,2,3,4,5],
                "date_parse": (-12,-4),
                "dtypes": {'Security Code':"object", 'Dividend yield (%)':"float", 'Dividend year':"Int32", 'P/E ratio':"float",
       'P/B ratio':"float", 'Fiscal year / quarter':"object", 'file_date': pd.Timestamp}

        },

        "stockquotes" : {
                        "globpath" : "./all_data/taiwan-stockquotes-data-website-cleaned-010122-10312022/*",
                        "cols_to_drop" : ['Unnamed: 15'],
                        "skiprows" : 0,
                        "skipfooter": 0,
                        "date_parse": (-33,-25),
                        "usecols": None,
                        "dtypes" : {'Security Code':"object", 'Trade Volume':"Int64", 'Transaction':"Int64", 'Trade Value':"Int64",
                                'Opening Price':"float", 'Highest Price':"float", 'Lowest Price':"float", 'Closing Price':"float",
                                'Dir(+/-)':"object", 'Change':"float", 'Last Best Bid Price':"float", 'Last Best Bid Volume':"Int64",
                                'Last Best Ask Price':"float", 'Last Best Ask Volume':"Int64", 'Price-Earning ratio':"float"
                                }
                        
        },
}

In [197]:
def make_frame(name):
    files = glob.glob(info[name]["globpath"])
    frames = []
    
    for file in files:
        try: 
            df = pd.read_csv(file, 
                            skiprows=info[name]["skiprows"], 
                            skipfooter=info[name]["skipfooter"], 
                            encoding_errors='replace', 
                            usecols=info[name]["usecols"], 
                            na_values=['-','--'],
                            thousands=',',
                            dtype=info[name]["dtypes"]
                        )
            df['file_date'] = pd.Timestamp(file[info[name]["date_parse"][0]:info[name]["date_parse"][1]])
            frames.append(df)

        except:
            print(f"Error reading csv file: {file}")
            continue
    
    df = pd.concat(frames, axis=0)
    
    if info[name]["cols_to_drop"] is not None:
        df.drop(columns=info[name]["cols_to_drop"], inplace=True)
    
    #extract security code
    df['Security Code Clean'] = df['Security Code'].str.extract(r'(\d{4,}[a-zA-Z]*)')

    #try to convert to optimal dtypes
    df.convert_dtypes()

    df.to_parquet(name+"_01012022_10312022")
    return df
    

In [106]:
###CLEAN STOCKQUOTES DATA
#regex to locate thousands separators
p = re.compile(r'(?<=\d),(?=\d)')

#iterate through stockquotes files, remove thousands separators, and save as new files
for filepath in stockquotes_files:
    oldpath = os.path.split(filepath)
    with open(filepath, 'r') as orig:
        new_path = os.path.join(os.path.join(os.getcwd(), "all_data", "taiwan-stockquotes-data-website-cleaned-010122-10312022"), oldpath[1][:-4]+"_cleaned.csv")
        with open(new_path, 'w') as new:
            for line in orig:
                new.write(re.subn(p,"",line)[0])

In [200]:
stockquotes_df = make_frame("stockquotes")

In [201]:
stockquotes_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 235301 entries, 0 to 1175
Data columns (total 17 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   Security Code         235301 non-null  object        
 1   Trade Volume          235301 non-null  Int64         
 2   Transaction           235301 non-null  Int64         
 3   Trade Value           235301 non-null  Int64         
 4   Opening Price         232340 non-null  float64       
 5   Highest Price         232340 non-null  float64       
 6   Lowest Price          232340 non-null  float64       
 7   Closing Price         232340 non-null  float64       
 8   Dir(+/-)              125784 non-null  object        
 9   Change                235301 non-null  float64       
 10  Last Best Bid Price   234952 non-null  float64       
 11  Last Best Bid Volume  235292 non-null  Int64         
 12  Last Best Ask Price   233834 non-null  float64       
 13  L

In [202]:
stockquotes_df

,Security Code,Trade Volume,Transaction,Trade Value,Opening Price,Highest Price,Lowest Price,Closing Price,Dir(+/-),Change,Last Best Bid Price,Last Best Bid Volume,Last Best Ask Price,Last Best Ask Volume,Price-Earning ratio,file_date,Security Code Clean
0,"=""0050""",7064552,8298,1034974917,146.00,147.35,146.00,146.40,+,0.90,146.35,32,146.40,96,0.00,2022-01-03,0050
1,"=""0051""",135357,185,8244702,60.90,61.30,60.75,60.90,+,0.10,60.75,2,60.90,45,0.00,2022-01-03,0051
2,"=""0052""",882163,488,119306921,134.65,135.95,134.65,135.20,+,1.05,135.20,54,135.35,7,0.00,2022-01-03,0052
3,"=""0053""",28349,25,1996804,70.25,70.75,70.25,70.45,+,0.50,70.40,20,70.45,1,0.00,2022-01-03,0053
4,"=""0054""",3998,13,127031,31.68,31.80,31.68,31.80,+,0.12,31.59,24,31.80,2,0.00,2022-01-03,0054
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1171,9944,71751,38,1459923,20.35,20.40,20.25,20.40,+,0.05,20.35,2,20.40,25,4.11,2022-10-31,9944
1172,9945,6956898,4827,276325555,40.30,40.50,39.25,39.60,NaN,0.40,39.60,93,39.65,5,5.34,2022-10-31,9945
1173,9946,11298,18,130561,11.45,11.80,11.45,11.55,+,0.05,11.50,2,11.55,1,18.33,2022-10-31,9946
1174,9955,131905,147,2244930,17.05,17.10,16.75,17.05,+,0.25,17.00,1,17.05,2,0.00,2022-10-31,9955


In [203]:
stockquotes = pd.read_parquet(r"C:\Users\uc8c\Desktop\daa.eq-service-prefunding-challenge\use_this_data\stockquotes_01012022_10312022")

In [204]:
stockquotes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 235301 entries, 0 to 1175
Data columns (total 17 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   Security Code         235301 non-null  object        
 1   Trade Volume          235301 non-null  Int64         
 2   Transaction           235301 non-null  Int64         
 3   Trade Value           235301 non-null  Int64         
 4   Opening Price         232340 non-null  float64       
 5   Highest Price         232340 non-null  float64       
 6   Lowest Price          232340 non-null  float64       
 7   Closing Price         232340 non-null  float64       
 8   Dir(+/-)              125784 non-null  object        
 9   Change                235301 non-null  float64       
 10  Last Best Bid Price   234952 non-null  float64       
 11  Last Best Bid Volume  235292 non-null  Int64         
 12  Last Best Ask Price   233834 non-null  float64       
 13  L

In [205]:
watchlist_df = make_frame("watchlist")

C:\Anaconda202105\envs\DataScience\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [206]:
watchlist_df

,Security Code,Periodic Call Auction Trading (see Remark below),file_date,Security Code Clean
0,1213,,2022-01-03,1213
1,1418,,2022-01-03,1418
2,1472,,2022-01-03,1472
3,1512,**,2022-01-03,1512
4,1538,,2022-01-03,1538
...,...,...,...,...
8,3043,**,2022-10-31,3043
9,3536,**,2022-10-31,3536
10,6225,**,2022-10-31,6225
11,8101,**,2022-10-31,8101


In [208]:
watchlist_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3440 entries, 0 to 12
Data columns (total 4 columns):
 #   Column                                            Non-Null Count  Dtype         
---  ------                                            --------------  -----         
 0   Security Code                                     3440 non-null   object        
 1   Periodic Call Auction Trading (see Remark below)  3440 non-null   object        
 2   file_date                                         3440 non-null   datetime64[ns]
 3   Security Code Clean                               3440 non-null   object        
dtypes: datetime64[ns](1), object(3)
memory usage: 134.4+ KB


In [209]:
watchlist = pd.read_parquet(r"C:\Users\uc8c\Desktop\daa.eq-service-prefunding-challenge\use_this_data\watchlist_01012022_10312022")

In [210]:
watchlist.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3440 entries, 0 to 12
Data columns (total 4 columns):
 #   Column                                            Non-Null Count  Dtype         
---  ------                                            --------------  -----         
 0   Security Code                                     3440 non-null   object        
 1   Periodic Call Auction Trading (see Remark below)  3440 non-null   object        
 2   file_date                                         3440 non-null   datetime64[ns]
 3   Security Code Clean                               3440 non-null   object        
dtypes: datetime64[ns](1), object(3)
memory usage: 134.4+ KB


In [224]:
watchlist

,Security Code,Periodic Call Auction Trading (see Remark below),file_date,Security Code Clean
0,1213,,2022-01-03,1213
1,1418,,2022-01-03,1418
2,1472,,2022-01-03,1472
3,1512,**,2022-01-03,1512
4,1538,,2022-01-03,1538
...,...,...,...,...
8,3043,**,2022-10-31,3043
9,3536,**,2022-10-31,3536
10,6225,**,2022-10-31,6225
11,8101,**,2022-10-31,8101


In [211]:
shortsales_df = make_frame("shortsales")

C:\Anaconda202105\envs\DataScience\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [212]:
shortsales_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 224945 entries, 0 to 1122
Data columns (total 5 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   Security Code        224945 non-null  object        
 1   Trading Volume       224945 non-null  Int64         
 2   Trading Value        224945 non-null  Int64         
 3   file_date            224945 non-null  datetime64[ns]
 4   Security Code Clean  224945 non-null  object        
dtypes: Int64(2), datetime64[ns](1), object(2)
memory usage: 10.7+ MB


In [255]:
shortsales_df

,Security Code,Trading Volume,Trading Value,file_date,Security Code Clean
0,"=""0050""",2,292950,2022-01-03,0050
1,"=""0051""",0,0,2022-01-03,0051
2,"=""0052""",0,0,2022-01-03,0052
3,"=""0053""",0,0,2022-01-03,0053
4,"=""0054""",0,0,2022-01-03,0054
...,...,...,...,...,...
1118,9944,0,0,2022-10-31,9944
1119,9945,173,6891050,2022-10-31,9945
1120,9946,0,0,2022-10-31,9946
1121,9955,0,0,2022-10-31,9955


In [221]:
shortsales_df = pd.read_parquet(r"C:\Users\uc8c\Desktop\daa.eq-service-prefunding-challenge\use_this_data\shortsales_01012022_10312022")

In [222]:
shortsales_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 224945 entries, 0 to 1122
Data columns (total 5 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   Security Code        224945 non-null  object        
 1   Trading Volume       224945 non-null  Int64         
 2   Trading Value        224945 non-null  Int64         
 3   file_date            224945 non-null  datetime64[ns]
 4   Security Code Clean  224945 non-null  object        
dtypes: Int64(2), datetime64[ns](1), object(2)
memory usage: 10.7+ MB


In [223]:
shortsales_df

,Security Code,Trading Volume,Trading Value,file_date,Security Code Clean
0,"=""0050""",2,292950,2022-01-03,0050
1,"=""0051""",0,0,2022-01-03,0051
2,"=""0052""",0,0,2022-01-03,0052
3,"=""0053""",0,0,2022-01-03,0053
4,"=""0054""",0,0,2022-01-03,0054
...,...,...,...,...,...
1118,9944,0,0,2022-10-31,9944
1119,9945,173,6891050,2022-10-31,9945
1120,9946,0,0,2022-10-31,9946
1121,9955,0,0,2022-10-31,9955


In [215]:
ratios_df = make_frame("ratios")

In [216]:
ratios_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 194392 entries, 0 to 964
Data columns (total 8 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   Security Code          194392 non-null  object        
 1   Dividend yield (%)     194392 non-null  float64       
 2   Dividend year          194392 non-null  Int32         
 3   P/E ratio              168174 non-null  float64       
 4   P/B ratio              194346 non-null  float64       
 5   Fiscal year / quarter  194392 non-null  object        
 6   file_date              194392 non-null  datetime64[ns]
 7   Security Code Clean    194392 non-null  object        
dtypes: Int32(1), datetime64[ns](1), float64(3), object(3)
memory usage: 12.8+ MB


In [219]:
ratios = pd.read_parquet(r'C:\Users\uc8c\Desktop\daa.eq-service-prefunding-challenge\use_this_data\ratios_01012022_10312022')

In [220]:
ratios

,Security Code,Dividend yield (%),Dividend year,P/E ratio,P/B ratio,Fiscal year / quarter,file_date,Security Code Clean
0,1101,7.10,2020,13.91,1.49,2021/3,2022-01-03,1101
1,1102,7.86,2020,10.09,1.03,2021/3,2022-01-03,1102
2,1103,6.76,2020,7.29,0.51,2021/3,2022-01-03,1103
3,1104,5.09,2020,12.78,0.75,2021/3,2022-01-03,1104
4,1108,5.88,2020,20.52,1.06,2021/3,2022-01-03,1108
...,...,...,...,...,...,...,...,...
960,9944,4.90,2021,4.11,0.66,2022/2,2022-10-31,9944
961,9945,17.68,2021,5.34,2.96,2022/2,2022-10-31,9945
962,9946,5.89,2021,18.33,0.59,2022/2,2022-10-31,9946
963,9955,0.00,2021,NaN,1.21,2022/2,2022-10-31,9955
